# 多相性 (Polymorphism)

1. パラメータ多相 (Parametric Polymorphism)
    ジェネリクスのようなもの

1. アドホック多相 (Ad-hoc Polymorphism)
    オブジェクト指向プログラミングでいうオーバーロード
    Haskellでは型クラスにより実現する。

# 型クラス

何らかの振る舞いを定義するインターフェース。  
OOPのオーバーロードみたいな機能を提供する。  

記述だけみると同一の式に見えても、型によって全く異なる実装の式を使い、型によって柔軟に処理を変えられます。（アドホック多相）  

型クラスは、二つの部分からなる。  

1. 型クラスのインスタンスになるために必要な契約を表わすクラスメソッドの型宣言。  
これに合うように、型クラスのインスタンスは、メソッドを実装しなくてはいけない。
1. デフォルトの実装。これは、インスタンスがクラスメソッドの型宣言で、指定した契約を独自に実装しなかったときに使われる。

型クラスは型や型クラスと同じ名前空間に存在する。  
型クラスのカインドはConstraint  

※ 型クラスのインスタンス指定は（他の言語と違って）型の定義時には行わない  

## カインド

型コンストラクタと型引数の組み合わせを規定する。

In [37]:
-- Int型
:k Int

Int :: *

In [38]:
-- Maybe a型
:k Maybe

Maybe :: * -> *

In [39]:
-- Functor型クラス
:k Functor

Functor :: (* -> *) -> Constraint

## 型クラス定義 class where


新しい型クラスを定義する構文
将来インスタンスになりえるあらゆる型の１つを表している。

In [45]:
class Eq a where
    -- 宣言部分。必須。
    (==) :: a -> a -> Bool
    (/=) :: a -> a -> Bool
    -- 実装部分。必須ではない。
    x == y = not (x /= y)
    x /= y = not (x == y)

Line 6: Use ==
Found:
not (x /= y)
Why not:
x == yLine 7: Use /=
Found:
not (x == y)
Why not:
x /= y

## インスタンス宣言 instance where

型を型クラスのインスタンスにする構文
インスタンス宣言時には型クラスが定義する関数（メソッド）を<u>**最小完全定義**</u>を満たすように実装する必要がある。

クラス定義

In [ ]:
class Eq a where

のa部分が実際の型で置換されている。

In [ ]:
instance Eq TrafficLight where
    Red == Red = True
    Green == Green = True
    Yellow == Yellow = True
    _ == _ = False

型クラスとインスタンスのサンプル

In [1]:
{-# LANGUAGE OverloadedStrings #-}

data Dog    = Dog deriving Show
data Cat    = Cat deriving Show
data Human  = Human String deriving Show


class Greeting a where
    name    :: a -> String
    hello   :: a -> String
    hello _ = "..." -- helloのデフォルト実装
    bye     :: a -> String
    bye   _ = "..." -- byeのデフォルト実装

instance Greeting Human where
    name (Human n)  = n
    hello h         = "Hi, I'm " ++ name h ++ "."
    bye _           = "See you."

instance Greeting Dog where
    name _  = "a dog"
    hello _ = "Bow!"

instance Greeting Cat where
    name _  = "a cat"
    hello _ = "Meow..."

hello (Human "Kenta")
hello Dog

"Hi, I'm Kenta."

"Bow!"

# 型

ある型クラスのインスタンス。  
型とは、値の集合についているラベル。

data, newtype, typeの3つの構文で型を定義する。

## 値


### 値コンストラクタ

引数をとってそのデータ型の値を返す関数。  
型コンストラクタ名と値コンストラクタ名は重複が可能なため、値コンストラクタが1つしかないデータ型はデータ型と値コンストラクタを同じ名前にするのが慣例。
（下の例ではPoint）

In [ ]:
data Point = Point Float Float Float deriving (Show)

型コンストラクタ名と値コンストラクタ名が異なる例

In [ ]:
data Employee = NewEmployee Integer Bool String deriving (Show)


型コンストラクタが複数ある場合

In [ ]:
data Bool = False | True

data Shape = Circle Float Float Float | Rectangle Float Float Float Float

data cmdOption =  COptInt Integer
                | COptBool Bool
                | COptStr String
                deriving (Show)

#### レコード記法

値コンストラクタの定義時に各引数に名前をつけることができます。

呼び出すときにはフィールドの名前を指定することが可能です。
フィールドはただの関数である



In [3]:
data Employee = NewEmployee {
                      age       :: Integer
                    , isManager :: Bool
                    , name      :: String
                } deriving (Show)

employee :: Employee
employee = NewEmployee {
                  name       = "Kenta Kozuka"
                , age        = 27
                , isManager  = False
            }

-- age :: Employee -> Integer
age employee

27

## 型宣言 data

新しい型を定義する構文。  
既存の型を別の新しい型で包む  
複数の値コンストラクタを持つデータ型を作ることができ、各値コンストラクタには0以上の個数のフィールドを持たせることが可能。

In [4]:
-- data 型コンストラクタ = 値コンストラクタ フィールドの型1 フィールドの型2 ...

data Point = Point Float Float Float deriving (Show)
data Bool = False | True

## newtype

「１つの型を取り、それを何かにくるんで別の型に見せかけたい」という場合に作られたもの。

* newtypeはdataよりも高速
* 値コンストラクタは1種類しか作れず、フィールドも1つだけ。

In [7]:
newtype ZipList a  = ZipList {
                        getZipList :: [a]
                    }

deriving も使用可能

In [6]:
newtype CharList = CharList {
                        getCharList :: [Char]
                    } deriving (Eq, Show)

Line 2: Use String
Found:
[Char]
Why not:
String

例）  
タプルをFunctorのインスタンスにして、fmapをタプルの第一要素を変更するようにしたい。

In [5]:
newtype Pair b a = Pair { getPair :: (a, b) }

instance Functor (Pair c) where
    fmap f (Pair (x, y)) = Pair (f x, y)

getPair $ fmap (*100) (Pair (2, 3))

(200,3)

## 型シノニム type

In [ ]:
newtype Pair b a = Pair { getPair :: (a, b) }

instance Functor (Pair c) where
    fmap f (Pair (x, y)) = Pair (f x, y)

Prelude> getPair $ fmap (*100) (Pair (2, 3))
(200, 3)

ある型に別の名前を与えて、コードドキュメントを他の人が読みやすくする。

In [ ]:
type String = [Char]

## 型コンストラクタ

型製造機
0個以上の具体型を受け取り、実際の値に割り当て可能な具体型を生成する。
型コンストラクタの引数は**型引数**と呼ばれる。

* 部分適用ができる
* 型コンストラクタはすべての引数を埋めて初めて新しい何かしらの値の型になる。
* （関数が値を引数にとって値を生み出すように）いずれは具体型になる
* だから、型クラスのインスタンスの型引数として、ある型を加工（部分適用）することができる。

In [ ]:
data Maybe a = Nothing | Just a

上記の例でいえば、Maybeが型コンストラクタで、
Just aが値コンストラクタ。
この型コンストラクタからMaybe Int, Maybe Car, Maybe Stringなどの型を作れる。
単なるMaybeという型の値は存在できない。なぜならMaybeは型コンストラクタであって、型ではない。

型コンストラクタが部分適用されている様子

In [48]:
:kind Either

Either :: * -> * -> *

In [50]:
:kind Either Int

Either Int :: * -> *

In [51]:
:kind Either Int String

Either Int String :: *

他の例として、 Functor 型クラスを見てみると、この型クラスのインスタンスになるためには、* -> * の種類の型コンストラクタでなくてはならないことがわかる。
Functorクラス宣言

In [ ]:
class Functor f where
    fmap :: (a -> b) -> f a -> f b

ここで、 f a は関数の型宣言にて値の型として使われていることから具体型でなくてはならない。
つまり、f は型引数を一つとることで具体型となる型コンストラクタであることがわかる。

In [46]:
Prelude> :info Num
class Num a where
  (+) :: a -> a -> a
  (-) :: a -> a -> a
  (*) :: a -> a -> a
  negate :: a -> a
  abs :: a -> a
  signum :: a -> a
  fromInteger :: Integer -> a
  {-# MINIMAL (+), (*), abs, signum, fromInteger, (negate | (-)) #-}
  	-- Defined in ‘GHC.Num’
instance Num Word -- Defined in ‘GHC.Num’
instance Num Integer -- Defined in ‘GHC.Num’
instance Num Int -- Defined in ‘GHC.Num’
instance Num Float -- Defined in ‘GHC.Float’
instance Num Double -- Defined in ‘GHC.Float’

## 種類

型の集合についているラベルのようなもの。

|種類|意味|例|
|---|---|---|
|*|具体型。具体型の型コンストラクタは型引数を取らない。値に付けられる型。|Int|
|* -> *|一つの具体型をとって具体型を返す型コンストラクタ|Maybe|
|* -> * -> *|二つの具体型をとって具体型を返す型コンストラクタ|Either|

### 具体型

型引数を取らない型の集合
値につけられる型は具体型だけ

* Int, Bool

data Bool = False | True

* Maybe Char や Maybe Bool

In [11]:
:t Just "hey"

Just "hey" :: Maybe [Char]

In [10]:
:t Just True

Just True :: Maybe Bool

## 型宣言

関数の型を明示的に与えること。

```
addTree :: Int -> Int -> Int -> Int
```

## 型変数

型安全を保ったまま、関数を複数の型に対して動作できるようにする。
以下のa

In [12]:
:t head

head :: forall a. [a] -> a

## 型シグネチャ

型を表す文字列のこと。
例えば、
```
quicksort :: (Ord a) => [a] -> [a]
```
の::以降
Haskellでは::の右側は型の世界。


## 型クラス制約

型シグネチャ宣言において型変数に与える制約。
=> の左辺
文脈 (context) とも呼ばれる。
上記の例では、「等値性関数は、同じ型の任意の2つの引数を取り、Boolを返す。<u>引数の2つの値の型はEqクラスのインスタンスでなければならない</u>」

```
(==) :: (Eq a) => a -> a -> Bool
```

## 型注釈

式が取るべき型をHaskellに明示的に教えてあげる手段。
式の終わりに

>::

を追記し、それから型を指定します。

In [13]:
read "5" :: Int

5

In [14]:
read "5" :: Float

5.0

# パターンマッチ

ある種のデータが従うべきパターンを指定し、そのパターンに従ってデータを分解するために使用する。

```
lucky :: Int -> String
lucky 7 = "Lucky Number Seven!"
lucky x = "Sorry, you're out of luck, pal!"
```

# ポイントフリースタイル

```
sum' :: (Num a) => [a] -> a
sum' xs = foldl (+) 0 xs
```
をポイントフリースタイルで書くと

```
sum' :: (Num a) => [a] -> a
sum' = foldl (+) 0
```
となる。
関数の定義を書いていると考えればよい。

*ポイントとは
```
fn x = f (g x)
```
のような関数定義に登場する一時変数xのことです。このポイントを使わないで関数を定義するスタイルなので、ポイントフリースタイル。

# レコード構文

# 多相型

型変数を用いた関数は多相的関数

# either

# ioアクション

# deriving

# Monoid モノイド

# as

# セクション


# ファンクタ、アプリカティブ・ファンクタ、モナド

* functorは Functor 型クラスを実装したデータ型。
* applicativeは Applicative 型クラスを実装したデータ型。
* モナドは Monad 型クラスを実装したデータ型。

Maybe はこの3つをすべて実装しており、functorでありapplicativeでありモナドでもある。

この3つの違いは何か?

<img src="https://camo.qiitausercontent.com/39a9d575095ac6384bde8cf4a6419b62b7bb498b/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f3838392f38656461363532662d313839662d326432322d636133662d6363393065366537373337642e706e67" style="width:800px" height="300px" />



* functor: fmap や <$> を使って、箱の中の値に関数を適用できる
* applicative: liftA や <*> を使って、箱のなかの関数を箱の中の値に適用できる
* モナド: liftM や >>= を使って、箱に入った値を返す関数を箱の中の値に適用できる


# 複雑系モナド

## Stetaモナドなしの場合

In [19]:
add = calc (+)
sub = calc (-)
mul = calc (*)
cdiv = calc div


bind :: (Stack -> (Integer, Stack)) -> (Integer -> (Stack -> (Integer, Stack))) -> (Stack -> (Integer, Stack))
bind opstack1 opstack2 = 
    \s -> let (i1, s1) = opstack1 s
    in (opstack2 i1) s1

poppp :: Stack -> (Integer, Stack)
poppp = pop `bind` \_ -> pop `bind` \_ -> pop

pushshop :: Stack -> (Integer, Stack)
pushshop = (push 1) `bind` \_ -> (push 2) `bind` \_ -> pop

calc :: (Integer -> Integer -> Integer) -> Stack -> (Integer, Stack)
calc op = pop `bind` \i1 -> pop `bind` \i2 -> push (op i1 i2)

type Stack = [Integer]

push :: Integer -> Stack -> (Integer, Stack)
push c cs = (c, c:cs)

pop :: Stack -> (Integer, Stack)
pop (c:cs) = (c, cs)

-- run
let stack = [1..5]
-- this should be (3, [4, 5])
poppp stack

(3,[4,5])

## Stateモナドを使った場合

bindは>>=と同じ働き

```
Monad.State 型クラス
    get = state $ \s -> (s, s) -- 現在の状態を取ってくる
    -- 状態型の引数を取り、「その状態を現在の状態に上書きする状態付き関数」を返す
    put newsState = state $ \s -> ((), newState)
```

In [28]:
import Control.Monad.State

add = calc (+)
sub = calc (-)
mul = calc (*)
cdiv = calc div

type Op = State [Integer] Integer

--  push c = do
--      cs <- get
--      put (c:cs)
--      return c
push :: Integer -> Op
push c = get >>= \cs -> put (c:cs) >>= \_ -> return c

--  pop = do
--      cs <- get
--      put (tail cs)
--      return
--  
pop :: Op
pop = get >>= \cs -> put (tail cs) >>= \_ -> return (head cs)

--  calc op = do
--      i1 <- pop
--      i2 <- pop
--      push (op i1 i2)
calc :: (Integer -> Integer -> Integer) -> Op
calc op = pop >>= \i1 -> pop >>= \i2 -> push (op i1 i2)

--  poppp = do pop;pop;pop
poppp :: Op
poppp = pop >>= \_ -> pop >>= \_ -> pop

--  pushshop = do push 1; push 2; pop
pushshop :: Op
pushshop = (push 1) >>= \_ -> (push 2) >>= \_ -> pop

runState poppp [1..5]

(3,[4,5])